In [22]:
import pandas as pd
import numpy as np
from scipy.stats import norm
import sympy as sym
from scipy.optimize import fsolve
r = 0.05
T = 1

In [23]:
stock = pd.read_excel("climate_transition_risk.xlsx", sheet_name=0)
company = pd.read_excel("climate_transition_risk.xlsx", sheet_name=1)
climate = pd.read_excel("climate_transition_risk.xlsx", sheet_name=2)

In [24]:
equity_vol = stock["Close Price"].std()
e_vol_1y = equity_vol / np.sqrt(10)
print(e_vol_1y)

3.700952038353894


In [25]:
last_price = stock["Close Price"].iloc[-1]
last_outst = company["Shares Outstanding (Millions)"].iloc[-1]
E = last_outst * last_price
D = 0.5*company["Long Term Debt (Millions)"].iloc[-1] + company["Short Term Debt (Millions)"].iloc[-1]

In [30]:
def dplus(A,a_vol):
    return (np.log(A/D) + (r+(a_vol**2))*T)/(a_vol*np.sqrt(T))
def dminus(A,a_vol):
    return (np.log(A/D) + (r-(a_vol**2))*T)/(a_vol*np.sqrt(T))

print(norm.cdf(0.5))

0.6914624612740131


In [39]:
asset = E + D
def myFunction(z):
   A = z[0]
   vol_A = z[1]

   F = np.empty((2))
   F[0] = A*norm.cdf(dplus(A,vol_A))-np.exp(-r)*D*norm.cdf(dminus(A,vol_A))-E
   F[1] = A*vol_A*norm.cdf(dplus(A,vol_A))-E*e_vol_1y
   return F

zGuess = np.array([asset,e_vol_1y])
z = fsolve(myFunction,zGuess)
print(z)


[5.29691824e+05 3.70067217e+00]


In [43]:
test1 = E*e_vol_1y
test2 = z[0]*z[1]*norm.cdf(dplus(z[0],z[1]))
test3 = E
test4 = A*norm.cdf(dplus(A,vol_A))-np.exp(-r)*D*norm.cdf(dminus(A,vol_A))-E
print(test1,test2)
print(test3,test4)
print(z[0],A,z[1],e_vol_1y)

1960214.645365714 1960214.645365714
529691.8240137097 537268.4586116 3.700672172881885 3.700952038353894
